In [1]:
import pandas as pd

In [4]:
raiz_db = '/home/eduardo/repositorios/IS_Agro/databases'
raiz_R = '/home/eduardo/Documentos/R'

planilha_agropecuaria = (pd.ExcelFile(f'{raiz_R}/Uso da Terra/outputs/extrapola_censo2017_MAPBIOMAS_V9-pastnat_2.1wet2.2grass.xlsx'))
planilha_rev_censo2017 = (pd.ExcelFile(f'{raiz_R}/Uso da Terra/outputs/Rextrap_censo2017MapbiomasV9_br-pastnat_2.1wet2.2grass.xlsx'))

In [50]:
anos = (
    pd.read_excel(planilha_agropecuaria, sheet_name=0, usecols='S:AI')
    .columns
    .to_list()
)

df_1 = (
    pd.read_excel(planilha_agropecuaria,
                  sheet_name=0,
                  usecols='C, S:AI',
                  skiprows=[21, 22, 23, 24]
                  )
    .drop(index=[0, 1, 2, 3, 4, 7, 8, 9, 13, 14, 15, 16, 17, 18, 19])
    .melt(
        id_vars='class_level_3',
        value_vars=anos,
        var_name='data',
        value_name='area'
        )
    .rename(columns=
        {
            'class_level_3':'label'
            })
    .assign(data = lambda dt: pd.to_datetime(dt.data, format='%Y')) 
    .convert_dtypes()
)

In [51]:
anos_extrapcenso = (
    pd.read_excel(planilha_rev_censo2017, sheet_name=0, usecols='E:K')
    .columns
    .to_list()
)

df_2 = (
    pd.read_excel(planilha_rev_censo2017, sheet_name=0, usecols='C, E:K')
    .melt(
        id_vars='cl_mapbiomas',
        value_vars=anos_extrapcenso,
        var_name='data',
        value_name='area'
    )
    .rename(columns=
        {
            'cl_mapbiomas':'label'
            })
    .assign(data = lambda dt: pd.to_datetime(dt.data, format='%Y')) 
    .convert_dtypes()
)

In [56]:
df_agropecuaria_final = (
    pd.concat([df_1, df_2])
    .reset_index(drop=True)
    .assign(country=1)
    .convert_dtypes()
    .set_index(['country', 'data', 'label'])
)

In [57]:
df_agropecuaria_final

area
country data       label                                  
1       2000-01-01 2.1. Wetland            21029731.474599
                   2.2. Grassland          28014504.135945
                   3.1. Pasture           180439684.157276
                   3.2.1. Temporary Crop   38620348.040022
                   3.2.2. Perennial Crop     1410015.78039
...                                                    ...
        2023-01-01 3.2.1. Temporary Crop     127683.580376
                   2.1_2.2                 44891073.081969
                   3.1. Pasture           106688663.075584
                   3.1. Pasture            12617087.050316
                   3.1. Pasture            14744626.521753

[134 rows x 1 columns]